In [1]:
from sentence_transformers import SentenceTransformer, util
import json
import torch
import numpy as np
import random
from tqdm.notebook import tqdm
torch.set_printoptions(sci_mode=False)
# import os
%config Completer.use_jedi = False

In [ ]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1').cuda()
temp = 0.03

### Caption - label similarity

In [ ]:
# def get_accuracy(caption_embeddings, class_embeddings, topk=5):
#     similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)
#     pseudoLabel = similarity_matrix.topk(topk, 1).indices    
#     gt_label = torch.Tensor([id_to_classid[fid] for fid in all_flickrids]).view(-1,1).repeat(1,topk)
#     matched_labels = (torch.Tensor(pseudoLabel) == torch.Tensor(gt_label))
#     top1_acc = matched_labels[:,0].sum()/len(gt_label)
#     topk_acc = matched_labels.any(1).sum()/len(gt_label)
#     return top1_acc, topk_acc

# def get_confident(caption_embeddings, class_embeddings, thres=.0025):
#     similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)
#     similarity_matrix = torch.nn.functional.softmax(similarity_matrix, dim=-1)
#     maxVal, pseudoLabel = similarity_matrix.max(1)
#     valid_inds = maxVal > thres
    
    
#     gt_label = torch.Tensor([id_to_classid[fid] for fid in all_flickrids])
#     matched_labels = (torch.Tensor(pseudoLabel) == torch.Tensor(gt_label))
#     matched_labels = matched_labels[valid_inds]
#     top1_acc = matched_labels.sum()/len(valid_inds)
# #     topk_acc = matched_labels.any(1).sum()/len(gt_label)
#     return top1_acc, sum(valid_inds)/len(similarity_matrix)*100

In [ ]:
## category
meta = json.load(open("../metadata/geoimnet.json"))

cats = {int(c["category_id"]):c["category_name"] for c in meta["categories"]}
# cats = [cats[idx].replace("_indoor","").replace("_outdoor","").replace("_"," ") for idx in range(205)]
cats = [cats[idx].replace("+"," ").replace("_"," ") for idx in range(600)]
# cats = ["A photo of a {}".format(c) for c in cats]
class_embeddings = model.encode(cats)

for dom in ["usa", "asia"]:# = "usa"

    id_to_cap = {m["image_id"]:m["llm_cap_llama_13b"] for m in meta["{}_train".format(dom)]["metadata"]}

    all_flickrids = list(id_to_cap.keys())
    all_captions = [id_to_cap[v] for v in all_flickrids]
    caption_embeddings = model.encode(all_captions)

    similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)

    soft_labels = torch.nn.functional.softmax(torch.Tensor(similarity_matrix)/temp, dim=-1).numpy()

    # pseudoLabel = similarity_matrix.argmax(1)

    write_str = ""
    with open("../soft_labels/imnet_{}_softPl.txt".format(dom), "w") as fh:
        for pl, fid in tqdm(zip(soft_labels, all_flickrids)):
            write_str += f"{fid} "
            vals = list(map(str, pl.tolist()))
            write_str += " ".join(vals)
            write_str += "\n"
        fh.write(write_str)

## Places

In [ ]:
## category
meta = json.load(open("../metadata/geoplaces.json"))

cats = {int(c["category_id"]):c["category_name"] for c in meta["categories"]}
cats = [cats[idx].replace("_indoor","").replace("_outdoor","").replace("_"," ") for idx in range(205)]
# cats = [cats[idx].replace("+"," ").replace("_"," ") for idx in range(600)]
# cats = ["A photo of a {}".format(c) for c in cats]
class_embeddings = model.encode(cats)

for dom in ["asia"]:# = "usa"

    id_to_cap = {m["image_id"]:m["llm_cap_llama_13b"] for m in meta["{}_train".format(dom)]["metadata"]}

    all_flickrids = list(id_to_cap.keys())
    all_captions = [id_to_cap[v] for v in all_flickrids]
    caption_embeddings = model.encode(all_captions)

    similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)

    soft_labels = torch.nn.functional.softmax(torch.Tensor(similarity_matrix)/temp, dim=-1).numpy()

    # pseudoLabel = similarity_matrix.argmax(1)

    write_str = ""
    with open("../soft_labels/places_{}_softPl.txt".format(dom), "w") as fh:
        for pl, fid in tqdm(zip(soft_labels, all_flickrids)):
            write_str += f"{fid} "
            vals = list(map(str, pl.tolist()))
            write_str += " ".join(vals)
            write_str += "\n"
        fh.write(write_str)

In [ ]:
len(pseudoLabel)

In [2]:
## examples

In [3]:
data = json.load(open("../metadata/geoplaces.json"))['usa_train']

In [4]:
id_to_meta = {m["image_id"]:m for m in data["metadata"]}
id_to_category = {m["image_id"]:m for m in data["annotations"]}

In [19]:
random_idx = random.sample(list(id_to_meta.keys()), 1)[0]
print(id_to_meta[random_idx])
print(id_to_category[random_idx])

{'image_id': 3802322729, 'datetaken': '2009-07-11 17:55:42', 'caption': 'Midwest fields', 'description': '© This photograph is copyrighted. Under no circumstances can it be reproduced, distributed, modified, copied, posted to websites or printed or published in media or other medium or used for commercial or other uses without the prior written consent and permission of the photographer.', 'tags': 'gold,wheat,fields,farm,farmer,midwest,kansas,nebraska,summer,us,usa,rural,america,pammorris,denverpam,nikon,d40', 'url': 'https://live.staticflickr.com/3532/3802322729_c67c8440e4.jpg', 'llm_cap_llama_13b': 'golden wheat fields in the Midwest.', 'blip1_cap': 'a field full of hay bales with trees in the background', 'blip2_cap': 'many round bales in the field with trees in the background'}
{'image_id': 3802322729, 'category': 201, 'class_name': 'wheat_field'}


In [10]:
random_idx

[125666]